# This notebook is concerned with running various ML models on chest imaging reports, and various aspects of its implementation

In [ ]:
# Generic imports
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import json
from pathlib import Path
from tqdm import tqdm   # For keeping track of loops
from joblib import Parallel, delayed
import sys

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

In [ ]:
# Custom imports
from custom_functions import *

sys.path.append("../")
import src.plots as plots

In [ ]:
# set plotting params
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
plt.style.reload_library()
rcparams = plots.stdrcparams1()
mpl.rcParams.update(rcparams)

In [ ]:
# Models/algorithms/classifiers
from xgboost import XGBClassifier

# Evaluation of models
from sklearn.model_selection import KFold
from sklearn.metrics import *

# Text vectorizer
from sklearn.feature_extraction.text import CountVectorizer

import shap

Read in data: Hospital A (2013), Hospital A (2016), Hospital B (2017-18), and MIMIC III.

In [ ]:
# Data locations
basedir = Path("../..")
training_path = basedir / "Analysis_Data" / "train_ML"
train_data1a = training_path / 'hospital_a_2013_bi_data_processed_minus_history_plus_conclusion-03-2021.csv'
train_data1b = training_path / 'hospital_a_2013_cxr_annotated.csv'

train_data2 = training_path / 'hospital_a_2016_bi_data_processed_minus_history_plus_conclusion.csv'

test_data3a = training_path / 'hospital_b_2017_cxr_annotated.csv'
test_data3b = training_path / 'hospital_b_2017_cxr_annotated_2023_jesse_curt_merged.csv'
test_data3c = training_path / 'hospital_b_2017_cxr_annotated_2023_three_annotators_merged.csv'
test_data3_whole = basedir / "Analysis_Data" / "hospital_b_2017" / 'cxr.csv'

whole_cxr_training = training_path / "cxr_whole_training_dataset.csv"

mimic3_path = basedir / "Analysis_Data" / "MIMIC_III" / "labeled_subset"

In [ ]:
# Figures
figure_path = basedir / "Figures"

### Hospital A (2013) read in and minor processing

In [ ]:
# Adjust which to read in desired Hospital A (2013) CXR reports
# 'a' is for originally processed file
# 'b' is for file processed by pipeline
which = 'b'
cols = {
        'a': ['segmented_report', 'score'],
        'b': ['seg_cxr_text', 'cxr_score']
        }

In [ ]:
if which == 'a':
    # open Hospital A (2013) processed files - segmented_reports
    segmented = pd.read_csv(train_data1a)
    segmented = segmented.drop(columns='Unnamed: 0').drop_duplicates()
    
elif which == 'b':
    # open Hospital A (2013) processed files - segmented_reports
    segmented = pd.read_csv(train_data1b)
    
else:
    raise ValueError("Type either 'a' or 'b', lower case.")

In [ ]:
# Removing remaining punctuation marks.
segmented[cols[which][0]] = segmented[cols[which][0]].str.replace(r"'", r"", regex=True)
segmented[cols[which][0]] = segmented[cols[which][0]].str.replace(r"\[", r"", regex=True)
segmented[cols[which][0]] = segmented[cols[which][0]].str.replace(r"]", r"", regex=True)
segmented[cols[which][0]] = segmented[cols[which][0]].str.replace(r",", r"", regex=True)

### Hospital A (2016) read in and minor processing

In [ ]:
segmented_cohort2 = pd.read_csv(train_data2)

# Replace some remaining punctuation marks
segmented_cohort2["segmented_report"] = segmented_cohort2["segmented_report"].str.replace(r"'", r"", regex=True)
segmented_cohort2["segmented_report"] = segmented_cohort2["segmented_report"].str.replace(r"\[", r"", regex=True)
segmented_cohort2["segmented_report"] = segmented_cohort2["segmented_report"].str.replace(r"\]", r"", regex=True)
segmented_cohort2["segmented_report"] = segmented_cohort2["segmented_report"].str.replace(r",", r"", regex=True)
segmented_cohort2 = segmented_cohort2.drop(columns='Unnamed: 0').drop_duplicates()

### Hospital B (2017-2018) read in and minor processing

In [ ]:
segmented_cohort3_original = pd.read_csv(test_data3a)

# Removing remaining punctuation marks
segmented_cohort3_original['seg_cxr_text'] = segmented_cohort3_original['seg_cxr_text'].str.replace(r"'", r"", regex=True)
segmented_cohort3_original['seg_cxr_text'] = segmented_cohort3_original['seg_cxr_text'].str.replace(r"\[", r"", regex=True)
segmented_cohort3_original['seg_cxr_text'] = segmented_cohort3_original['seg_cxr_text'].str.replace(r"\]", r"", regex=True)
segmented_cohort3_original['seg_cxr_text'] = segmented_cohort3_original['seg_cxr_text'].str.replace(r",", r"", regex=True)

### CXR whole training dataset read in

In [ ]:
training_data = pd.read_csv(whole_cxr_training)

# Removing remaining punctuation marks
training_data['seg_cxr_text'] = training_data['seg_cxr_text'].str.replace(r"'", r"", regex=True)
training_data['seg_cxr_text'] = training_data['seg_cxr_text'].str.replace(r"\[", r"", regex=True)
training_data['seg_cxr_text'] = training_data['seg_cxr_text'].str.replace(r"\]", r"", regex=True)
training_data['seg_cxr_text'] = training_data['seg_cxr_text'].str.replace(r",", r"", regex=True)

### Generating bootstrapped AUCs, calibration, and feature importances

In [ ]:
n_boot = 100   # Make smaller if wanting the notebook to run faster.
n_boot1 = 10

XG_test_auc = []
XG_tprs = []
XG_fops = []
XG_preds = []
XG_rsquared = []
XG_dws = []
XG_importances = []
XG_shap = []
XG_test_set = []

LR_test_auc = []
LR_tprs = []
LR_fops = []
LR_preds = []
LR_rsquared = []
LR_dws = []
LR_importances = []
LR_shap = []
LR_test_set = []

RF_test_auc = []
RF_tprs = []
RF_fops = []
RF_preds = []
RF_rsquared = []
RF_dws = []
RF_importances = []
RF_shap = []
RF_test_set = []

DT_test_auc = []
DT_tprs = []
DT_fops = []
DT_preds = []
DT_rsquared = []
DT_dws = []
DT_importances = []
DT_shap = []
DT_test_set = []

mean_fpr = np.arange(0, 1.01, 0.01)
mean_mpv = np.arange(0, 1.1, 0.1)

In [ ]:
# A lot of graphs generated by this notebook depend on this loop, including the calibration curve
# (which is part of the panel where the interrater agreement graph is)
for i in tqdm(range(n_boot1)):

    # This line resamples the data, WITH replacement
    boot_segmented = training_data.sample(n=len(training_data), replace=True, axis=0)
    
    encounters = boot_segmented['encounter_id'].unique()
    cv = KFold()
    
    # # Accounting for potential label imbalance
    # count_0 = pd.Series(y_train).value_counts()[0]
    # count_1 = pd.Series(y_train).value_counts()[1]
    # weight_0 = count_0 / len(y_train)
    # weight_1 = count_1 / len(y_train)
    # print(f" %No: {weight_0*100:.2f}%, %Yes: {weight_1*100:.2f}%")
    
    
    # Training each model
    print("Working on XGBoost")
    if __name__ == '__main__':
        output1, output2, output3, output4, output5 = Parallel(n_jobs=15)(delayed(nested_cv)(
            boot_segmented,
            cols,
            which,
            encounters,
            train_index,
            test_index,
            mean_fpr,
            mean_mpv,
            model="XGBoost",
            score='log_loss'
            ) for train_index, test_index in cv.split(encounters))
    
    XG_test_auc.append(np.mean([output1[0], output2[0], output3[0], output4[0], output5[0]]))
    XG_preds.extend(output1[1])
    XG_preds.extend(output2[1])
    XG_preds.extend(output3[1])
    XG_preds.extend(output4[1])
    XG_preds.extend(output5[1])
    XG_tprs.append(np.mean([output1[2], output2[2], output3[2], output4[2], output5[2]], axis=0))
    XG_fops.append(np.mean([output1[3], output2[3], output3[3], output4[3], output5[3]], axis=0))
    XG_rsquared.append(np.mean([output1[4], output2[4], output3[4], output4[4], output5[4]], axis=0))
    XG_dws.append(np.mean([output1[5], output2[5], output3[5], output4[5], output5[5]], axis=0))
    XG_importances.extend(pd.DataFrame(
        output1[6]+output2[6]+output3[6]+output4[6]+output5[6]).groupby('feature').mean().reset_index().to_dict(orient='records'))
    

    print("Working on Logistic Regression")
    if __name__ == '__main__':
        output1, output2, output3, output4, output5 = Parallel(n_jobs=15)(delayed(nested_cv)(
            boot_segmented,
            cols,
            which,
            encounters,
            train_index,
            test_index,
            mean_fpr,
            mean_mpv,
            model="LogisticRegression",
            score='log_loss'
            ) for train_index, test_index in cv.split(encounters))
    
    LR_test_auc.append(np.mean([output1[0], output2[0], output3[0], output4[0], output5[0]]))
    LR_preds.extend(output1[1])
    LR_preds.extend(output2[1])
    LR_preds.extend(output3[1])
    LR_preds.extend(output4[1])
    LR_preds.extend(output5[1])
    LR_tprs.append(np.mean([output1[2], output2[2], output3[2], output4[2], output5[2]], axis=0))
    LR_fops.append(np.mean([output1[3], output2[3], output3[3], output4[3], output5[3]], axis=0))
    LR_rsquared.append(np.mean([output1[4], output2[4], output3[4], output4[4], output5[4]], axis=0))
    LR_dws.append(np.mean([output1[5], output2[5], output3[5], output4[5], output5[5]], axis=0))
    LR_importances.extend(pd.DataFrame(
        output1[6]+output2[6]+output3[6]+output4[6]+output5[6]).groupby('feature').mean().reset_index().to_dict(orient='records'))


    print("Working on Random Forest")
    if __name__ == '__main__':
        output1, output2, output3, output4, output5 = Parallel(n_jobs=15)(delayed(nested_cv)(
            boot_segmented,
            cols,
            which,
            encounters,
            train_index,
            test_index,
            mean_fpr,
            mean_mpv,
            model="RandomForest",
            score='log_loss'
            ) for train_index, test_index in cv.split(encounters))
    
    RF_test_auc.append(np.mean([output1[0], output2[0], output3[0], output4[0], output5[0]]))
    RF_preds.extend(output1[1])
    RF_preds.extend(output2[1])
    RF_preds.extend(output3[1])
    RF_preds.extend(output4[1])
    RF_preds.extend(output5[1])
    RF_tprs.append(np.mean([output1[2], output2[2], output3[2], output4[2], output5[2]], axis=0))
    RF_fops.append(np.mean([output1[3], output2[3], output3[3], output4[3], output5[3]], axis=0))
    RF_rsquared.append(np.mean([output1[4], output2[4], output3[4], output4[4], output5[4]], axis=0))
    RF_dws.append(np.mean([output1[5], output2[5], output3[5], output4[5], output5[5]], axis=0))
    RF_importances.extend(pd.DataFrame(
        output1[6]+output2[6]+output3[6]+output4[6]+output5[6]).groupby('feature').mean().reset_index().to_dict(orient='records'))


    print("Working on Decision Tree")
    if __name__ == '__main__':
        output1, output2, output3, output4, output5 = Parallel(n_jobs=15)(delayed(nested_cv)(
            boot_segmented,
            cols,
            which,
            encounters,
            train_index,
            test_index,
            mean_fpr,
            mean_mpv,
            model="DecisionTree",
            score='log_loss'
            ) for train_index, test_index in cv.split(encounters))
        
    DT_test_auc.append(np.mean([output1[0], output2[0], output3[0], output4[0], output5[0]]))
    DT_preds.extend(output1[1])
    DT_preds.extend(output2[1])
    DT_preds.extend(output3[1])
    DT_preds.extend(output4[1])
    DT_preds.extend(output5[1])
    DT_tprs.append(np.mean([output1[2], output2[2], output3[2], output4[2], output5[2]], axis=0))
    DT_fops.append(np.mean([output1[3], output2[3], output3[3], output4[3], output5[3]], axis=0))
    DT_rsquared.append(np.mean([output1[4], output2[4], output3[4], output4[4], output5[4]], axis=0))
    DT_dws.append(np.mean([output1[5], output2[5], output3[5], output4[5], output5[5]], axis=0))
    DT_importances.extend(pd.DataFrame( 
        output1[6]+output2[6]+output3[6]+output4[6]+output5[6]).groupby('feature').mean().reset_index().to_dict(orient='records'))

#### Modifying data collected in the loop, for plotting

In [ ]:
# Decision Tree
mean_DT_tpr = np.mean(DT_tprs, axis=0)
mean_DT_tpr[-1] = 0.0
DT_tpr_CI95 = [
        np.percentile(DT_tprs, 2.5, axis=0),
        np.percentile(DT_tprs, 97.5, axis=0)
        ]

mean_DT_auc = np.mean(DT_test_auc)
DT_auc_CI95 = [
        np.percentile(DT_test_auc, 2.5),
        np.percentile(DT_test_auc, 97.5)
        ]

DT_importances_df = pd.DataFrame(DT_importances).groupby('feature').agg(
    mean_imp=('importance', np.mean),
    low_CI=('importance', lambda x: x.mean() - x.quantile(0.025)),
    high_CI=('importance', lambda x: x.quantile(0.975) - x.mean())).reset_index().sort_values(
            by='mean_imp',
            ascending=False
            )

mean_DT_fop = np.mean(DT_fops, axis=0)
DT_fop_CI95 = [
        np.percentile(DT_fops, 2.5, axis=0),
        np.percentile(DT_fops, 97.5, axis=0)
        ]
mean_DT_rsquared = np.mean(DT_rsquared, axis=0)
DT_rsquared_CI95 = [
        np.percentile(DT_rsquared, 2.5, axis=0),
        np.percentile(DT_rsquared, 97.5, axis=0)
        ]
mean_DT_dws = np.mean(DT_dws, axis=0)
DT_dws_CI95 = [
        np.percentile(DT_dws, 2.5, axis=0),
        np.percentile(DT_dws, 97.5, axis=0)
        ]


# Logistic Regression
mean_LR_tpr = np.mean(LR_tprs, axis=0)
mean_LR_tpr[-1] = 0.0
LR_tpr_CI95 = [
        np.percentile(LR_tprs, 2.5, axis=0),
        np.percentile(LR_tprs, 97.5, axis=0)
        ]

mean_LR_auc = np.mean(LR_test_auc)
LR_auc_CI95 = [
        np.percentile(LR_test_auc, 2.5),
        np.percentile(LR_test_auc, 97.5)
        ]

LR_importances_df = pd.DataFrame(LR_importances).groupby('feature').agg(
    mean_imp=('importance', np.mean),
    low_CI=('importance', lambda x: x.mean() - x.quantile(0.025)),
    high_CI=('importance', lambda x: x.quantile(0.975) - x.mean())).reset_index().sort_values(
            by='mean_imp',
            ascending=False,
            key=np.absolute
            )

mean_LR_fop = np.mean(LR_fops, axis=0)
LR_fop_CI95 = [
        np.percentile(LR_fops, 2.5, axis=0),
        np.percentile(LR_fops, 97.5, axis=0)
        ]
mean_LR_rsquared = np.mean(LR_rsquared, axis=0)
LR_rsquared_CI95 = [
        np.percentile(LR_rsquared, 2.5, axis=0),
        np.percentile(LR_rsquared, 97.5, axis=0)
        ]
mean_LR_dws = np.mean(LR_dws, axis=0)
LR_dws_CI95 = [
        np.percentile(LR_dws, 2.5, axis=0),
        np.percentile(LR_dws, 97.5, axis=0)
        ]


# Random Forest
mean_RF_tpr = np.mean(RF_tprs, axis=0)
mean_RF_tpr[-1] = 0.0
RF_tpr_CI95 = [
        np.percentile(RF_tprs, 2.5, axis=0),
        np.percentile(RF_tprs, 97.5, axis=0)
        ]

mean_RF_auc = np.mean(RF_test_auc)
RF_auc_CI95 = [
        np.percentile(RF_test_auc, 2.5),
        np.percentile(RF_test_auc, 97.5)
        ]

RF_importances_df = pd.DataFrame(RF_importances).groupby('feature').agg(
    mean_imp=('importance', np.mean),
    low_CI=('importance', lambda x: x.mean() - x.quantile(0.025)),
    high_CI=('importance', lambda x: x.quantile(0.975) - x.mean())).reset_index().sort_values(
            by='mean_imp',
            ascending=False
            )

mean_RF_fop = np.mean(RF_fops, axis=0)
RF_fop_CI95 = [
        np.percentile(RF_fops, 2.5, axis=0),
        np.percentile(RF_fops, 97.5, axis=0)
        ]
mean_RF_rsquared = np.mean(RF_rsquared, axis=0)
RF_rsquared_CI95 = [
        np.percentile(RF_rsquared, 2.5, axis=0),
        np.percentile(RF_rsquared, 97.5, axis=0)
        ]
mean_RF_dws = np.mean(RF_dws, axis=0)
RF_dws_CI95 = [
        np.percentile(RF_dws, 2.5, axis=0),
        np.percentile(RF_dws, 97.5, axis=0)
        ]


# XGBoost
mean_XG_tpr = np.mean(XG_tprs, axis=0)
mean_XG_tpr[-1] = 0.0
XG_tpr_CI95 = [
        np.percentile(XG_tprs, 2.5, axis=0),
        np.percentile(XG_tprs, 97.5, axis=0)
        ]

mean_XG_auc = np.mean(XG_test_auc)
XG_auc_CI95 = [
        np.percentile(XG_test_auc, 2.5),
        np.percentile(XG_test_auc, 97.5)
        ]

XG_importances_df = pd.DataFrame(XG_importances).groupby('feature').agg(
    mean_imp=('importance', np.mean),
    low_CI=('importance', lambda x: x.mean() - x.quantile(0.025)),
    high_CI=('importance', lambda x: x.quantile(0.975) - x.mean())).reset_index().sort_values(
            by='mean_imp',
            ascending=False
            )

mean_XG_fop = np.mean(XG_fops, axis=0)
XG_fop_CI95 = [
        np.percentile(XG_fops, 2.5, axis=0),
        np.percentile(XG_fops, 97.5, axis=0)
        ]
mean_XG_rsquared = np.mean(XG_rsquared, axis=0)
XG_rsquared_CI95 = [
        np.percentile(XG_rsquared, 2.5, axis=0),
        np.percentile(XG_rsquared, 97.5, axis=0)
        ]
mean_XG_dws = np.mean(XG_dws, axis=0)
XG_dws_CI95 = [
        np.percentile(XG_dws, 2.5, axis=0),
        np.percentile(XG_dws, 97.5, axis=0)
        ]

# For AUC
models = ['Decision\nTree', 'Logistic\nRegression', 'Random\nForest', 'XGBoost']
heights = [mean_DT_auc, mean_LR_auc, mean_RF_auc, mean_XG_auc]
CI95 = [
        [
                heights[0] - DT_auc_CI95[0],
                heights[1] - LR_auc_CI95[0],
                heights[2] - RF_auc_CI95[0],
                heights[3] - XG_auc_CI95[0]
        ],
        [
                DT_auc_CI95[1] - heights[0],
                LR_auc_CI95[1] - heights[1],
                RF_auc_CI95[1] - heights[2],
                XG_auc_CI95[1] - heights[3]
        ]
       ]

### Plotting ROC curves with uncertainties

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))

ax.plot([0, 1], [0, 1], linestyle="--", color="k", alpha=0.8)

ax.plot(mean_fpr, mean_DT_tpr, color='#66c2a5', label=f"{heights[0]:.2f} ({DT_auc_CI95[0]:.2f}-{DT_auc_CI95[1]:.2f})")
ax.fill_between(mean_fpr, DT_tpr_CI95[0], DT_tpr_CI95[1],
                color='#66c2a5', alpha=0.2)

ax.plot(mean_fpr, mean_LR_tpr, color='#fc8d62', label=f"{heights[1]:.2f} ({LR_auc_CI95[0]:.2f}-{LR_auc_CI95[1]:.2f})")
ax.fill_between(mean_fpr, LR_tpr_CI95[0], LR_tpr_CI95[1],
                color='#fc8d62', alpha=0.2)

ax.plot(mean_fpr, mean_RF_tpr, color='#8da0cb', label=f"{heights[2]:.2f} ({RF_auc_CI95[0]:.2f}-{RF_auc_CI95[1]:.2f})")
ax.fill_between(mean_fpr, RF_tpr_CI95[0], RF_tpr_CI95[1],
                color='#8da0cb', alpha=0.2)

ax.plot(mean_fpr, mean_XG_tpr, color='#e78ac3', label=f"{heights[3]:.2f} ({XG_auc_CI95[0]:.2f}-{XG_auc_CI95[1]:.2f})")
ax.fill_between(mean_fpr, XG_tpr_CI95[0], XG_tpr_CI95[1],
                color='#e78ac3', alpha=0.2)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax.grid(linestyle=':')
ax.legend(loc='best', frameon=False)

plt.tight_layout()
# plt.savefig(figure_path / "fig1_ROC_curves_all_models.pdf", dpi=1000)
plt.show()

In [ ]:
fig1, ax1 = plt.subplots(figsize=(7,7))

ax1.plot(mean_fpr, mean_DT_tpr, color='#66c2a5', label=f"Decision tree performance:\n{mean_DT_auc:.2f}")
ax1.fill_between(mean_fpr, DT_tpr_CI95[0], DT_tpr_CI95[1],
                 color='#66c2a5', alpha=0.2)

ax1.plot([0, 1], [0, 1], linestyle="--", color="k", alpha=0.8)

ax1.set_ylabel("True Positive Rate")
ax1.set_xlabel("False Positive Rate")
# ax1.set_title("Decision Tree")
ax1.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax1.grid(linestyle=':')

plt.tight_layout()
# plt.savefig(figure_path / "ROC_curves_DT.pdf", dpi=1000)
plt.show()

In [ ]:
fig2, ax2 = plt.subplots(figsize=(7,7))

ax2.plot(mean_fpr, mean_LR_tpr, color='#fc8d62', label=f"Logistic regression performance:\n{mean_LR_auc:.2f}")
ax2.fill_between(mean_fpr, LR_tpr_CI95[0], LR_tpr_CI95[1],
                 alpha=0.2, color='#fc8d62')
ax2.plot([0, 1], [0, 1], linestyle="--", color="k", alpha=0.8)

ax2.set_ylabel("True Positive Rate")
ax2.set_xlabel("False Positive Rate")
# ax2.set_title("Logistic Regression")
ax2.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax2.grid(linestyle=':')

plt.tight_layout()
# plt.savefig(figure_path / "ROC_curves_LR.pdf", dpi=1000)
plt.show()

In [ ]:
fig3, ax3 = plt.subplots(figsize=(7,7))

ax3.plot(mean_fpr, mean_RF_tpr, color='#8da0cb', label=f"Random Forest performance:\n{mean_RF_auc:.2f}")
ax3.fill_between(mean_fpr, RF_tpr_CI95[0], RF_tpr_CI95[1],
                 color='#8da0cb', alpha=0.2)
ax3.plot([0, 1], [0, 1], linestyle="--", color="k", alpha=0.8)

ax3.set_ylabel("True Positive Rate")
ax3.set_xlabel("False Positive Rate")
# ax3.set_title("Random Forest")
ax3.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax3.grid(linestyle=':')

plt.tight_layout()
# plt.savefig(figure_path / "ROC_curves_RF.pdf", dpi=1000)
plt.show()

In [ ]:
fig4, ax4 = plt.subplots(figsize=(7,7))

ax4.plot(mean_fpr, mean_XG_tpr, color='#e78ac3', label=f"XGBoost performance:\n{mean_XG_auc:.2f}")
ax4.fill_between(mean_fpr, XG_tpr_CI95[0], XG_tpr_CI95[1],
                 color='#e78ac3', alpha=0.2)
ax4.plot([0, 1], [0, 1], linestyle="--", color="k", alpha=0.8)

ax4.set_ylabel("True Positive Rate")
ax4.set_xlabel("False Positive Rate")
# ax4.set_title("XGBoost")
ax4.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax4.grid(linestyle=':')

plt.tight_layout()
# plt.savefig(figure_path / "ROC_curves_XG.pdf", dpi=1000)
plt.show()

In [ ]:
fig5, ax5 = plt.subplots(figsize=plots.stdfigsize())
ax5.bar(models, heights, color=['#66c2a5', '#fc8d62', '#8da0cb', '#e78ac3'],
        yerr=CI95, capsize=5)

ax5.set_ylabel('Performance (AUROC)')
ax5.set_xlabel('')
ax5.grid(linestyle=':', axis='y')
ax5.set_ylim(0., .5)
plt.tight_layout()
# plt.savefig(figure_path / 'fig1_auc_different_models.pdf', dpi=1000)
# plt.savefig(figure_path / 'fig1_auc_different_models.png', dpi=1000)
plt.show()

###  Plotting feature importances

In [ ]:
imp_words = ['edema', 'pulmonary edema', 'bilateral', 'atelectasis', 'diffuse', 'interstitial', 'no', 'clear']

In [ ]:
LR1 = LR_importances_df.nlargest(7,
                                 columns="mean_imp",
                                 keep='all')

LR2 = LR_importances_df.nsmallest(7,
                                  columns="mean_imp",
                                  keep='all').sort_values(by='mean_imp', ascending=False)

LR_imp = pd.concat([LR1, LR2])

In [ ]:
fig6, ax6 = plt.subplots(figsize=plots.stdfigsize())

DT_importances_df.nlargest(15,
                           columns="mean_imp",
                           keep='all').sort_values(by='mean_imp').plot(x='feature',
                                                                       y='mean_imp',
                                                                       kind='barh',
                                                                       color='#66c2a5',
                                                                       ax=ax6,
                                                                       capsize=4,
                                                                       xerr=[list(DT_importances_df.nlargest(15,
                                                                                                             columns="mean_imp",
                                                                                                             keep='all')['low_CI']),
                                                                             list(DT_importances_df.nlargest(15,
                                                                                                             columns="mean_imp",
                                                                                                             keep='all')['high_CI'])],
                                                                       label='Decision Tree')

for tick in ax6.yaxis.get_major_ticks():
     if tick.label1.get_text() in imp_words:
         tick.label1.set_fontweight('bold')
     else:
         pass
ax6.set_xlabel('Importance')
ax6.set_ylabel('')
ax6.grid(linestyle=':', axis='x')
ax6.legend(loc='best', frameon=False)
# ax6.set_title("Decision Tree")
plt.tight_layout()
# plt.savefig(figure_path / 'fig1_importance_DT.pdf', dpi=1000)
plt.show()

In [ ]:
fig7, ax7 = plt.subplots(figsize=plots.stdfigsize())

LR_imp.plot(x='feature',
            y='mean_imp',
            kind='barh',
            color='#fc8d62',
            ax=ax7,
            capsize=4,
            xerr=[list(LR_imp.nlargest(15,
                                       columns="mean_imp",
                                       keep='all')['low_CI']),
                  list(LR_imp.nlargest(15,
                                       columns="mean_imp",
                                       keep='all')['high_CI'])],
            label="Logistic\nRegression")

for tick in ax7.yaxis.get_major_ticks():
     if tick.label1.get_text() in imp_words:
         tick.label1.set_fontweight('bold')
     else:
         pass
ax7.set_xlabel('Importance')
ax7.set_ylabel('')
ax7.grid(linestyle=':', axis='x')
ax7.legend(loc='best', frameon=False)
# ax7.set_title("Logistic Regression")
plt.tight_layout()
# plt.savefig(figure_path /'fig1_importance_LR.pdf', dpi=1000)
plt.show()

In [ ]:
fig8, ax8 = plt.subplots(figsize=plots.stdfigsize())

RF_importances_df.nlargest(15,
                           columns="mean_imp",
                           keep='all').sort_values(by='mean_imp').plot(x='feature',
                                                                       y='mean_imp',
                                                                       kind='barh',
                                                                       color='#8da0cb',
                                                                       ax=ax8,
                                                                       capsize=4,
                                                                       xerr=[list(RF_importances_df.nlargest(15,
                                                                                                             columns="mean_imp",
                                                                                                             keep='all')['low_CI']),
                                                                             list(RF_importances_df.nlargest(15,
                                                                                                             columns="mean_imp",
                                                                                                             keep='all')['high_CI'])],
                                                                       label="Random Forest")

for tick in ax8.yaxis.get_major_ticks():
     if tick.label1.get_text() in imp_words:
         tick.label1.set_fontweight('bold')
     else:
         pass
ax8.set_xlabel('Importance')
ax8.grid(linestyle=':', axis='x')
ax8.set_ylabel('')
ax8.legend(loc='best', frameon=False)
# ax8.set_title("Random Forest")
plt.tight_layout()
# plt.savefig(figure_path / 'fig1_importance_RF.pdf', dpi=1000)
plt.show()

In [ ]:
fig9, ax9 = plt.subplots(figsize=plots.stdfigsize())

XG_importances_df.nlargest(15,
                           columns="mean_imp",
                           keep='all').sort_values(by='mean_imp').plot(x='feature',
                                                                       y='mean_imp',
                                                                       kind='barh',
                                                                       color='#e78ac3',
                                                                       ax=ax9,
                                                                       capsize=4,
                                                                       xerr=[list(XG_importances_df.nlargest(15,
                                                                                                             columns="mean_imp",
                                                                                                             keep='all')['low_CI']),
                                                                             list(XG_importances_df.nlargest(15,
                                                                                                             columns="mean_imp",
                                                                                                             keep='all')['high_CI'])],
                                                                       label="XGBoost")

for tick in ax9.yaxis.get_major_ticks():
     if tick.label1.get_text() in imp_words:
         tick.label1.set_fontweight('bold')
     else:
         pass
ax9.set_xlabel('Importance')
ax9.grid(linestyle=':', axis='x')
ax9.set_ylabel('')
ax9.legend(loc='best', frameon=False)
# ax9.set_title("XGBoost")
plt.tight_layout()
# plt.savefig(figure_path / 'fig1_importance_XG.pdf', dpi=1000)
plt.show()

In [ ]:
fig10, ax10 = plt.subplots(3, 2, figsize=plots.stdfigsize(nx=2, ny=3))

ax10[0,0].plot([0, 1], [0, 1], linestyle="--", color="k", alpha=0.8, label="No-skill line")

# ax10[0,0].plot(mean_fpr, mean_DT_tpr, color='#66c2a5', label=f"Decision tree performance:\n{mean_DT_auc:.2f}")
# ax10[0,0].fill_between(mean_fpr, DT_tpr_CI95[0], DT_tpr_CI95[1],
#                 color='#66c2a5', alpha=0.2)

# ax10[0,0].plot(mean_fpr, mean_LR_tpr, color='#fc8d62', label="Logistic regression")
# ax10[0,0].fill_between(mean_fpr, LR_tpr_CI95[0], LR_tpr_CI95[1],
#                 color='#fc8d62', alpha=0.2)

# ax10[0,0].plot(mean_fpr, mean_RF_tpr, color='#8da0cb', label="Random Forest")
# ax10[0,0].fill_between(mean_fpr, RF_tpr_CI95[0], RF_tpr_CI95[1],
#                 color='#8da0cb', alpha=0.2)

ax10[0,0].plot(mean_fpr, mean_XG_tpr, color='#e78ac3', label=f"XGBoost\nMean AUROC: {mean_XG_auc:.2f}")
ax10[0,0].fill_between(mean_fpr, XG_tpr_CI95[0], XG_tpr_CI95[1],
                color='#e78ac3', alpha=0.2)

ax10[0,0].set_xlabel("False Positive Rate")
ax10[0,0].set_ylabel("True Positive Rate")
ax10[0,0].set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax10[0,0].grid(linestyle=':')
ax10[0,0].legend(loc='lower right', frameon=False, fontsize=20)
ax10[0,0].text(-0.25, 1.05, "a", transform=ax10[0,0].transAxes,
               fontsize=30, fontweight='bold', va='top')


ax10[0,1].bar(models, heights, color=['#66c2a5', '#fc8d62', '#8da0cb', '#e78ac3'],
        yerr=CI95, capsize=5)

# ax10[0,1].set_xlabel('Classifier')
ax10[0,1].set_ylabel('Performance (AUROC)')
ax10[0,1].grid(linestyle=':', axis='y')
ax10[0,1].set_ylim(0.5, 1.0)
ax10[0,1].text(-0.25, 1.05, "b", transform=ax10[0,1].transAxes,
               fontsize=30, fontweight='bold', va='top')


DT_importances_df.nlargest(15,
                           columns="mean_imp",
                           keep='all').sort_values(by='mean_imp').plot(x='feature',
                                                                       y='mean_imp',
                                                                       kind='barh',
                                                                       color='#66c2a5',
                                                                       ax=ax10[1,0],
                                                                       capsize=4,
                                                                       xerr=[list(DT_importances_df.nlargest(15,
                                                                                                             columns="mean_imp",
                                                                                                             keep='all')['low_CI']),
                                                                             list(DT_importances_df.nlargest(15,
                                                                                                             columns="mean_imp",
                                                                                                             keep='all')['high_CI'])],
                                                                       label='Decision Tree')

for tick in ax10[1,0].yaxis.get_major_ticks():
     if tick.label1.get_text() in imp_words:
         tick.label1.set_fontweight('bold')
     else:
         pass
ax10[1,0].set_xlabel('Importance')
ax10[1,0].set_ylabel('')
ax10[1,0].grid(linestyle=':', axis='x')
ax10[1,0].legend(loc='best', frameon=False, markerscale=0.5, fontsize=20)
ax10[1,0].text(-0.30, 1.05, "c", transform=ax10[1,0].transAxes,
               fontsize=30, fontweight='bold', va='top')


LR_imp.plot(x='feature',
            y='mean_imp',
            kind='barh',
            color='#fc8d62',
            ax=ax10[1,1],
            capsize=4,
            xerr=[list(LR_imp.nlargest(15,
                                       columns="mean_imp",
                                       keep='all')['low_CI']),
                  list(LR_imp.nlargest(15,
                                       columns="mean_imp",
                                       keep='all')['high_CI'])],
            label="Logistic\nRegression")

for tick in ax10[1,1].yaxis.get_major_ticks():
     if tick.label1.get_text() in imp_words:
         tick.label1.set_fontweight('bold')
     else:
         pass
ax10[1,1].set_xlabel('Importance')
ax10[1,1].set_ylabel('')
ax10[1,1].grid(linestyle=':', axis='x')
ax10[1,1].legend(loc='best', frameon=False, markerscale=0.5)


RF_importances_df.nlargest(15,
                           columns="mean_imp",
                           keep='all').sort_values(by='mean_imp').plot(x='feature',
                                                                       y='mean_imp',
                                                                       kind='barh',
                                                                       color='#8da0cb',
                                                                       ax=ax10[2,0],
                                                                       capsize=4,
                                                                       xerr=[list(RF_importances_df.nlargest(15,
                                                                                                             columns="mean_imp",
                                                                                                             keep='all')['low_CI']),
                                                                             list(RF_importances_df.nlargest(15,
                                                                                                             columns="mean_imp",
                                                                                                             keep='all')['high_CI'])],
                                                                       label="Random Forest")
                           
for tick in ax10[2,0].yaxis.get_major_ticks():
     if tick.label1.get_text() in imp_words:
         tick.label1.set_fontweight('bold')
     else:
         pass
ax10[2,0].set_xlabel('Importance')
ax10[2,0].grid(linestyle=':', axis='x')
ax10[2,0].set_ylabel('')
ax10[2,0].legend(loc='best', frameon=False, markerscale=0.5)


XG_importances_df.nlargest(15,
                           columns="mean_imp",
                           keep='all').sort_values(by='mean_imp').plot(x='feature',
                                                                       y='mean_imp',
                                                                       kind='barh',
                                                                       color='#e78ac3',
                                                                       ax=ax10[2,1],
                                                                       capsize=4,
                                                                       xerr=[list(XG_importances_df.nlargest(15,
                                                                                                             columns="mean_imp",
                                                                                                             keep='all')['low_CI']),
                                                                             list(XG_importances_df.nlargest(15,
                                                                                                             columns="mean_imp",
                                                                                                             keep='all')['high_CI'])],
                                                                       label="XGBoost")

for tick in ax10[2,1].yaxis.get_major_ticks():
     if tick.label1.get_text() in imp_words:
         tick.label1.set_fontweight('bold')
     else:
         pass
ax10[2,1].set_xlabel('Importance')
ax10[2,1].grid(linestyle=':', axis='x')
ax10[2,1].set_ylabel('')
ax10[2,1].legend(loc='best', frameon=False, markerscale=0.5)

fig10.tight_layout()
# plt.savefig(figure_path / 'fig1.png', dpi=1000)
# plt.savefig(figure_path / 'fig1.pdf', dpi=1000)
plt.show()

###  Plotting calibration curves with uncertainties

In [ ]:
fig11, ax11 = plt.subplots(2, 2, figsize=plots.stdfigsize(nx=2, ny=2))

# Logistic Regression
ax11[0,0].plot(
    mean_mpv, mean_DT_fop, color='#66c2a5', marker="o",
    label=f"Decision Tree\nMean DW = {mean_DT_dws:.2f}")
ax11[0,0].fill_between(mean_mpv, DT_fop_CI95[0], DT_fop_CI95[1], color='#66c2a5', alpha=0.2)

# Logistic Regression
ax11[0,1].plot(
    mean_mpv, mean_LR_fop, color='#fc8d62', marker="o",
    label=f"Logistic Regression\nMean DW = {mean_LR_dws:.2f}")
ax11[0,1].fill_between(mean_mpv, LR_fop_CI95[0], LR_fop_CI95[1], color='#fc8d62', alpha=0.2)

# Random Forest
ax11[1,0].plot(
    mean_mpv, mean_RF_fop, color='#8da0cb', marker="o",
    label=f"Random Forest\nMean DW = {mean_RF_dws:.2f}")
ax11[1,0].fill_between(mean_mpv,RF_fop_CI95[0], RF_fop_CI95[1], color='#8da0cb', alpha=0.2)

# XGBoost
ax11[1,1].plot(
    mean_mpv, mean_XG_fop, color='#e78ac3', marker="o",
    label=f"XGBoost\nMean DW = {mean_XG_dws:.2f}")
ax11[1,1].fill_between(mean_mpv, XG_fop_CI95[0], XG_fop_CI95[1], color='#e78ac3', alpha=0.2)

# Plot properties
ax11[0,0].plot(mean_mpv, mean_mpv, linestyle="--", color="k", alpha=0.8, label="Perfectly calibrated")
ax11[0,0].set_ylabel("Fraction of positive labels")
ax11[0,0].set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax11[0,0].grid(linestyle=':')
ax11[0,0].legend(loc='best', frameon=False)
ax11[0,0].text(-0.2, 1.05, "a", transform=ax11[0,0].transAxes,
               fontsize=30, fontweight='bold', va='top')

ax11[0,1].plot(mean_mpv, mean_mpv, linestyle="--", color="k", alpha=0.8)
ax11[0,1].set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax11[0,1].grid(linestyle=':')
ax11[0,1].legend(loc='best', frameon=False)
ax11[0,1].text(-0.2, 1.05, "b", transform=ax11[0,1].transAxes,
               fontsize=30, fontweight='bold', va='top')

ax11[1,0].plot(mean_mpv, mean_mpv, linestyle="--", color="k", alpha=0.8)
ax11[1,0].set_ylabel("Fraction of positive labels")
ax11[1,0].set_xlabel("Mean predicted probability")
ax11[1,0].set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax11[1,0].grid(linestyle=':')
ax11[1,0].legend(loc='best', frameon=False)
ax11[1,0].text(-0.2, 1.05, "c", transform=ax11[1,0].transAxes,
               fontsize=30, fontweight='bold', va='top')

ax11[1,1].plot(mean_mpv, mean_mpv, linestyle="--", color="k", alpha=0.8)
ax11[1,1].set_xlabel("Mean predicted probability")
ax11[1,1].set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax11[1,1].grid(linestyle=':')
ax11[1,1].legend(loc='best', frameon=False)
ax11[1,1].text(-0.2, 1.05, "d", transform=ax11[1,1].transAxes,
               fontsize=30, fontweight='bold', va='top')

plt.tight_layout()
# plt.savefig(figure_path / "fig2.pdf", dpi=1000)
# plt.savefig(figure_path / "fig2.png", dpi=1000)
plt.show()

In [ ]:
figu, eje = plt.subplots(2, 2, figsize=(15,15))

# Decision Tree
eje[0,0].hist(DT_preds, bins=20)

eje[0,0].tick_params(axis='x', labelsize=18)
eje[0,0].tick_params(axis='y', labelsize=18)
eje[0,0].set_ylabel("Counts", size=18)
eje[0,0].set_title("Decision Tree", size=20)
eje[0,0].grid(linestyle=':')
eje[0,0].set(ylim=[0, 225000])

# Logistic Regression
eje[0,1].hist(LR_preds, bins=20)

eje[0,1].tick_params(axis='x', labelsize=18)
eje[0,1].tick_params(axis='y', labelsize=18)
eje[0,1].set_title("Logistic Regression", size=20)
eje[0,1].grid(linestyle=':')
eje[0,1].set(ylim=[0, 225000])

# Random Forest
eje[1,0].hist(RF_preds, bins=20)

eje[1,0].tick_params(axis='x', labelsize=18)
eje[1,0].tick_params(axis='y', labelsize=18)
eje[1,0].set_ylabel("Counts", size=18)
eje[1,0].set_xlabel("Estimated probabilities", size=18)
eje[1,0].set_title("Random Forest", size=20)
eje[1,0].grid(linestyle=':')
eje[1,0].set(ylim=[0, 225000])

# XGBoost
eje[1,1].hist(XG_preds, bins=20)

eje[1,1].tick_params(axis='x', labelsize=18)
eje[1,1].tick_params(axis='y', labelsize=18)
eje[1,1].set_xlabel("Estimated probabilities", size=18)
eje[1,1].set_title("XGBoost", size=20)
eje[1,1].grid(linestyle=':')
eje[1,1].set(ylim=[0, 225000])

plt.tight_layout()
# plt.savefig(figure_path / "probability_histograms.png", dpi=1000)
plt.show()

## Generalizability figure

In [ ]:
# Reading in Hospital B (2017-18)
segmented_cohort3_jesse_curt = pd.read_csv(test_data3b)
segmented_cohort3_three_annot = pd.read_csv(test_data3c)

segmented_cohort3_jesse_curt['seg_cxr_text'] = segmented_cohort3_jesse_curt['seg_cxr_text'].str.replace(r"'", r"", regex=True)
segmented_cohort3_jesse_curt['seg_cxr_text'] = segmented_cohort3_jesse_curt['seg_cxr_text'].str.replace(r"\[", r"", regex=True)
segmented_cohort3_jesse_curt['seg_cxr_text'] = segmented_cohort3_jesse_curt['seg_cxr_text'].str.replace(r"\]", r"", regex=True)
segmented_cohort3_jesse_curt['seg_cxr_text'] = segmented_cohort3_jesse_curt['seg_cxr_text'].str.replace(r",", r"", regex=True)

segmented_cohort3_three_annot['seg_cxr_text'] = segmented_cohort3_three_annot['seg_cxr_text'].str.replace(r"'", r"", regex=True)
segmented_cohort3_three_annot['seg_cxr_text'] = segmented_cohort3_three_annot['seg_cxr_text'].str.replace(r"\[", r"", regex=True)
segmented_cohort3_three_annot['seg_cxr_text'] = segmented_cohort3_three_annot['seg_cxr_text'].str.replace(r"\]", r"", regex=True)
segmented_cohort3_three_annot['seg_cxr_text'] = segmented_cohort3_three_annot['seg_cxr_text'].str.replace(r",", r"", regex=True)

In [ ]:
# Reading in MIMIC III
mimic_iii = pd.read_csv(mimic3_path / "cxr.csv")

# Removing remaining punctuation marks
mimic_iii['seg_cxr_text'] = mimic_iii['seg_cxr_text'].str.replace(r"'", r"", regex=True)
mimic_iii['seg_cxr_text'] = mimic_iii['seg_cxr_text'].str.replace(r"\[", r"", regex=True)
mimic_iii['seg_cxr_text'] = mimic_iii['seg_cxr_text'].str.replace(r"\]", r"", regex=True)
mimic_iii['seg_cxr_text'] = mimic_iii['seg_cxr_text'].str.replace(r",", r"", regex=True)

In [ ]:
# Hospital A (2013)
X1 = segmented[cols[which][0]].to_numpy()
Y1 = segmented[cols[which][1]].to_numpy()

# # Accounting for potential label imbalance
# count_0_Y1 = pd.Series(Y1).value_counts()[0]
# count_1_Y1 = pd.Series(Y1).value_counts()[1]
# weight_0_Y1 = count_0_Y1/ len(Y1)
# weight_1_Y1 = count_1_Y1 / len(Y1)

# Hospital A (2016)
X2 = segmented_cohort2['segmented_report'].to_numpy()
Y2 = segmented_cohort2['score'].to_numpy()

# # Accounting for potential label imbalance
# count_0_Y2 = pd.Series(Y2).value_counts()[0]
# count_1_Y2 = pd.Series(Y2).value_counts()[1]
# weight_0_Y2 = count_0_Y2 / len(Y2)
# weight_1_Y2 = count_1_Y2 / len(Y2)

# Hospital B (2017-18)
X3_original = segmented_cohort3_original['seg_cxr_text'].values
Y3_original = segmented_cohort3_original['score_final'].values
X3_jesse_curt = segmented_cohort3_jesse_curt['seg_cxr_text'].values
X3_three_annot = segmented_cohort3_three_annot['seg_cxr_text'].values

# MIMIC III
X4 = mimic_iii['seg_cxr_text'].to_numpy()
Y4 = mimic_iii['curt_bl_infiltrates_(1=yes)'].to_numpy()

which1 = 'a'
cols1 = {
    'a': ['seg_cxr_text', 'score_final'],
    'b': ['seg_cxr_text', 'score']
    }

which2 = 'a'
cols2 = {
    'a': ['seg_cxr_text', 'curt_bl_infiltrates_(1=yes)'],
    'b': ['seg_cxr_text', 'cxr_score_predicted']
    }

### Get optimized hyperparameters for each of the models

In [ ]:
with open(basedir / "Development_notebooks" /  "hyperparameters" / "XG_hyperparams_hospital_a_2013.json", "r") as xg_file:
    xg_hyperparams = json.load(xg_file)

xg_hyperparams['base_score'] = float(xg_hyperparams['base_score'])    
xg_hyperparams['n_estimators'] = int(xg_hyperparams['n_estimators'])
xg_hyperparams['max_depth'] = int(xg_hyperparams['max_depth'])
xg_hyperparams['learning_rate'] = float(xg_hyperparams['learning_rate'])
xg_hyperparams['gamma'] = float(xg_hyperparams['gamma'])
xg_hyperparams['min_child_weight'] = float(xg_hyperparams['min_child_weight'])
xg_hyperparams['max_delta_step'] = float(xg_hyperparams['max_delta_step'])
xg_hyperparams['subsample'] = float(xg_hyperparams['subsample'])


with open(basedir / "Development_notebooks" /  "hyperparameters" / "XG_hyperparams_hospital_a_2016.json", "r") as xg_file2:
    xg_hyperparams2 = json.load(xg_file2)

xg_hyperparams2['base_score'] = float(xg_hyperparams2['base_score']) 
xg_hyperparams2['n_estimators'] = int(xg_hyperparams2['n_estimators'])
xg_hyperparams2['max_depth'] = int(xg_hyperparams2['max_depth'])
xg_hyperparams2['learning_rate'] = float(xg_hyperparams2['learning_rate'])
xg_hyperparams2['gamma'] = float(xg_hyperparams2['gamma'])
xg_hyperparams2['min_child_weight'] = float(xg_hyperparams2['min_child_weight'])
xg_hyperparams2['max_delta_step'] = float(xg_hyperparams2['max_delta_step'])
xg_hyperparams2['subsample'] = float(xg_hyperparams2['subsample'])

### Train on Hospital A (2013)

In [ ]:
# Vectorize
vect1 = CountVectorizer(
    tokenizer=tokenizer_better,
    ngram_range=(1, 2),
    max_features=200
    )

vect1.fit(X1)
X1_vect_1 = vect1.transform(X1).toarray()

XG_model_1 = XGBClassifier(
    **xg_hyperparams,
    tree_method='hist',
    random_state=0
    )

XG_model_1.fit(X1_vect_1, Y1)

#### Test on Hospital A (2016)

In [ ]:
tprs_1_2 = []
aucs_1_2 = []
fops_1_2 = []
preds_1_2 = []

for i in tqdm(range(n_boot)):
    # This line resamples the data, WITH replacement
    boot_segmented_2 = segmented_cohort2.sample(
        n=len(segmented_cohort2),
        replace=True,
        axis=0
        )
    
    encounters = boot_segmented_2['_id'].unique()
    
    cv = KFold()
    output1, output2, output3, output4, output5 = Parallel(n_jobs=5)(delayed(custom_cv_not_train)(
        XG_model_1,
        boot_segmented_2,
        {'a': ['segmented_report', 'score']},
        'a',
        encounters,
        vect1,
        test_index,
        mean_fpr,
        mean_mpv
        ) for test_index, _ in cv.split(encounters))
    
    aucs_1_2.append(np.mean([output1[0], output2[0], output3[0], output4[0], output5[0]]))
    preds_1_2.extend(output1[1])
    preds_1_2.extend(output2[1])
    preds_1_2.extend(output3[1])
    preds_1_2.extend(output4[1])
    preds_1_2.extend(output5[1])
    tprs_1_2.append(np.mean([output1[2], output2[2], output3[2], output4[2], output5[2]], axis=0))
    fops_1_2.append(np.mean([output1[3], output2[3], output3[3], output4[3], output5[3]], axis=0))

#### Test on Hospital B (2017-18)

In [ ]:
tprs_1_3 = []
aucs_1_3 = []
fops_1_3 = []
preds_1_3 = []

for i in tqdm(range(n_boot)):
    # This line resamples the data, WITH replacement
    boot_segmented_3 = segmented_cohort3_original.sample(
        n=len(segmented_cohort3_original),
        replace=True,
        axis=0
        )
    
    encounters = boot_segmented_3['icu_id'].unique()
    
    cv = KFold()
    output1, output2, output3, output4, output5 = Parallel(n_jobs=5)(delayed(custom_cv_not_train)(
        XG_model_1,
        boot_segmented_3,
        cols1,
        which1,
        encounters,
        vect1,
        test_index,
        mean_fpr,
        mean_mpv
        ) for test_index, _ in cv.split(encounters))
    
    aucs_1_3.append(np.mean([output1[0], output2[0], output3[0], output4[0], output5[0]]))
    preds_1_3.extend(output1[1])
    preds_1_3.extend(output2[1])
    preds_1_3.extend(output3[1])
    preds_1_3.extend(output4[1])
    preds_1_3.extend(output5[1])
    tprs_1_3.append(np.mean([output1[2], output2[2], output3[2], output4[2], output5[2]], axis=0))
    fops_1_3.append(np.mean([output1[3], output2[3], output3[3], output4[3], output5[3]], axis=0))

#### Test on a labeled subset of MIMIC III

In [ ]:
tprs_1_mimic = []
aucs_1_mimic = []
fops_1_mimic = []
preds_1_mimic = []

for i in tqdm(range(n_boot)):
    # This line resamples the data, WITH replacement
    boot_mimic3 = mimic_iii.sample(
        n=len(mimic_iii),
        replace=True,
        axis=0
        )
    
    encounters = boot_mimic3['encounter_id'].unique()
    
    cv = KFold()
    output1, output2, output3, output4, output5 = Parallel(n_jobs=5)(delayed(custom_cv_not_train)(
        XG_model_1,
        boot_mimic3,
        cols2,
        which2,
        encounters,
        vect1,
        test_index,
        mean_fpr,
        mean_mpv
        ) for test_index, _ in cv.split(encounters))
    
    aucs_1_mimic.append(np.mean([output1[0], output2[0], output3[0], output4[0], output5[0]]))
    preds_1_mimic.extend(output1[1])
    preds_1_mimic.extend(output2[1])
    preds_1_mimic.extend(output3[1])
    preds_1_mimic.extend(output4[1])
    preds_1_mimic.extend(output5[1])
    tprs_1_mimic.append(np.mean([output1[2], output2[2], output3[2], output4[2], output5[2]], axis=0))
    fops_1_mimic.append(np.mean([output1[3], output2[3], output3[3], output4[3], output5[3]], axis=0))

### Train on Hospital A (2016)

In [ ]:
XG2_imp = []

In [ ]:
# Vectorize
vect2 = CountVectorizer(
    tokenizer=tokenizer_better,
    ngram_range=(1, 2),
    max_features=200
    )

vect2.fit(X2)
X2_vect_2 = vect2.transform(X2).toarray()
features2 = {value: key for key, value in vect2.vocabulary_.items()}

XG_model_2 = XGBClassifier(
    **xg_hyperparams2,
    tree_method='hist',
    random_state=0
    )
    
XG_model_2.fit(X2_vect_2, Y2)

In [ ]:
raw_imp_XG2 = XG_model_2.feature_importances_
for i in range(len(raw_imp_XG2)):
    temp = {'feature': features2[i], 'importance': raw_imp_XG2[i]}
    XG2_imp.append(temp)
    
XG2_imp_df = pd.DataFrame(XG2_imp).sort_values(by='importance', ascending=False)

In [ ]:
figu, eje1 = plt.subplots(figsize=plots.stdfigsize())

XG2_imp_df.nlargest(15,
                    columns="importance",
                    keep='all').sort_values(by='importance').plot(x='feature',
                                                                  y='importance',
                                                                  kind='barh',
                                                                  ax=eje1,
                                                                  legend=False)

eje1.set_xlabel('Normalized mean performance gain')
eje1.grid(linestyle=':', axis='x')
eje1.set_ylabel('')
eje1.set_title("Importances - XGBoost trained on Hospital A (2016)")
figu.tight_layout()
plt.show()

#### Test on Hospital A (2013)

In [ ]:
tprs_2_1 = []
aucs_2_1 = []
fops_2_1 = []
preds_2_1 = []

for i in tqdm(range(n_boot)):
    # This line resamples the data, WITH replacement
    boot_segmented_1 = segmented.sample(
        n=len(segmented),
        replace=True,
        axis=0
        )
    
    encounters = boot_segmented_1['encounter_id'].unique()
    
    cv = KFold()
    output1, output2, output3, output4, output5 = Parallel(n_jobs=5)(delayed(custom_cv_not_train)(
        XG_model_2,
        boot_segmented_1,
        cols,
        which,
        encounters,
        vect2,
        test_index,
        mean_fpr,
        mean_mpv
        ) for test_index, _ in cv.split(encounters))
    
    aucs_2_1.append(np.mean([output1[0], output2[0], output3[0], output4[0], output5[0]]))
    preds_2_1.extend(output1[1])
    preds_2_1.extend(output2[1])
    preds_2_1.extend(output3[1])
    preds_2_1.extend(output4[1])
    preds_2_1.extend(output5[1])
    tprs_2_1.append(np.mean([output1[2], output2[2], output3[2], output4[2], output5[2]], axis=0))
    fops_2_1.append(np.mean([output1[3], output2[3], output3[3], output4[3], output5[3]], axis=0))

#### Test on Hospital B (2017-18)

In [ ]:
tprs_2_3 = []
aucs_2_3 = []
fops_2_3 = []
preds_2_3 = []

for i in tqdm(range(n_boot)):
    # This line resamples the data, WITH replacement
    boot_segmented_3 = segmented_cohort3_original.sample(
        n=len(segmented_cohort3_original),
        replace=True,
        axis=0
        )
    
    encounters = boot_segmented_3['icu_id'].unique()

    cv = KFold()
    output1, output2, output3, output4, output5 = Parallel(n_jobs=5)(delayed(custom_cv_not_train)(
        XG_model_2,
        boot_segmented_3,
        cols1,
        which1,
        encounters,
        vect2,
        test_index,
        mean_fpr,
        mean_mpv
        ) for test_index, _ in cv.split(encounters))
    
    aucs_2_3.append(np.mean([output1[0], output2[0], output3[0], output4[0], output5[0]]))
    preds_2_3.extend(output1[1])
    preds_2_3.extend(output2[1])
    preds_2_3.extend(output3[1])
    preds_2_3.extend(output4[1])
    preds_2_3.extend(output5[1])
    tprs_2_3.append(np.mean([output1[2], output2[2], output3[2], output4[2], output5[2]], axis=0))
    fops_2_3.append(np.mean([output1[3], output2[3], output3[3], output4[3], output5[3]], axis=0))

#### Test on a labeled subset of MIMIC III

In [ ]:
tprs_2_mimic = []
aucs_2_mimic = []
fops_2_mimic = []
preds_2_mimic = []

for i in tqdm(range(n_boot)):
    # This line resamples the data, WITH replacement
    boot_mimic3 = mimic_iii.sample(
        n=len(mimic_iii),
        replace=True,
        axis=0
        )
    
    encounters = boot_mimic3['encounter_id'].unique()
    
    cv = KFold()
    output1, output2, output3, output4, output5 = Parallel(n_jobs=5)(delayed(custom_cv_not_train)(
        XG_model_2,
        boot_mimic3,
        cols2,
        which2,
        encounters,
        vect2,
        test_index,
        mean_fpr,
        mean_mpv
        ) for test_index, _ in cv.split(encounters))
    
    aucs_2_mimic.append(np.mean([output1[0], output2[0], output3[0], output4[0], output5[0]]))
    preds_2_mimic.extend(output1[1])
    preds_2_mimic.extend(output2[1])
    preds_2_mimic.extend(output3[1])
    preds_2_mimic.extend(output4[1])
    preds_2_mimic.extend(output5[1])
    tprs_2_mimic.append(np.mean([output1[2], output2[2], output3[2], output4[2], output5[2]], axis=0))
    fops_2_mimic.append(np.mean([output1[3], output2[3], output3[3], output4[3], output5[3]], axis=0))

## Plotting

In [ ]:
# Hospital A (2013) on Hospital A (2016)
mean_tpr_1_2 = np.mean(tprs_1_2, axis=0)
mean_tpr_1_2[-1] = 1.0
tpr_1_2_CI95 = [
    np.percentile(tprs_1_2, 25, axis=0),
    np.percentile(tprs_1_2, 75, axis=0)
    ]

mean_auc_1_2 = np.mean(aucs_1_2)
auc_1_2_CI95 = [
    np.percentile(aucs_1_2, 2.5),
    np.percentile(aucs_1_2, 97.5)
    ]


# Hospital A (2013) on Hospital B (2017-18)
mean_tpr_1_3 = np.mean(tprs_1_3, axis=0)
mean_tpr_1_3[-1] = 1.0
tpr_1_3_CI95 = [
    np.percentile(tprs_1_3, 2.5, axis=0),
    np.percentile(tprs_1_3, 97.5, axis=0)
    ]

mean_auc_1_3 = np.mean(aucs_1_3)
auc_1_3_CI95 = [
    np.percentile(aucs_1_3, 2.5),
    np.percentile(aucs_1_3, 97.5)
    ]


# Hospital A (2013) on MIMIC-III
mean_tpr_1_mimic = np.mean(tprs_1_mimic, axis=0)
mean_tpr_1_mimic[-1] = 1.0
tpr_1_mimic_CI95 = [
    np.percentile(tprs_1_mimic, 2.5, axis=0),
    np.percentile(tprs_1_mimic, 97.5, axis=0)
    ]

mean_auc_1_mimic = np.mean(aucs_1_mimic)
auc_1_mimic_CI95 = [
    np.percentile(aucs_1_mimic, 2.5),
    np.percentile(aucs_1_mimic, 97.5)
    ]



# Hospital A (2016) on Hospital A (2013)
mean_tpr_2_1 = np.mean(tprs_2_1, axis=0)
mean_tpr_2_1[-1] = 1.0
tpr_2_1_CI95 = [
    np.percentile(tprs_2_1, 2.5, axis=0),
    np.percentile(tprs_2_1, 97.5, axis=0)
    ]

mean_auc_2_1 = np.mean(aucs_2_1)
auc_2_1_CI95 = [
    np.percentile(aucs_2_1, 2.5),
    np.percentile(aucs_2_1, 97.5)
    ]


# Hospital A (2016) on Hospital B (2017-18)
mean_tpr_2_3 = np.mean(tprs_2_3, axis=0)
mean_tpr_2_3[-1] = 1.0
tpr_2_3_CI95 = [
    np.percentile(tprs_2_3, 2.5, axis=0),
    np.percentile(tprs_2_3, 97.5, axis=0)
    ]

mean_auc_2_3 = np.mean(aucs_2_3)
auc_2_3_CI95 = [
    np.percentile(aucs_2_3, 2.5),
    np.percentile(aucs_2_3, 97.5)
    ]

# Hospital A (2016) on MIMIC-III
mean_tpr_2_mimic = np.mean(tprs_2_mimic, axis=0)
mean_tpr_2_mimic[-1] = 1.0
tpr_2_mimic_CI95 = [
    np.percentile(tprs_2_mimic, 2.5, axis=0),
    np.percentile(tprs_2_mimic, 97.5, axis=0)
    ]

mean_auc_2_mimic = np.mean(aucs_2_mimic)
auc_2_mimic_CI95 = [
    np.percentile(aucs_2_mimic, 2.5),
    np.percentile(aucs_2_mimic, 97.5)
    ]

# For AUC plot
tests1 = [
    'Hospital B (2017-18)',
    'MIMIC (2001-12)'
    ]
heights1 = [mean_auc_1_3, mean_auc_1_mimic]
CI951 = [
    [heights1[0] - auc_1_3_CI95[0], heights1[1] - auc_1_mimic_CI95[0]],
    [auc_1_3_CI95[1] - heights1[0], auc_1_mimic_CI95[1] - heights1[1]]
]

tests2 = [
    'Hospital B (2017-18)',
    'MIMIC (2001-12)'
    ]
heights2 = [mean_auc_2_3, mean_auc_2_mimic]
CI952 = [
    [heights2[0] - auc_2_3_CI95[0], heights2[1] - auc_2_mimic_CI95[0]],
    [auc_2_3_CI95[1] - heights2[0], auc_2_mimic_CI95[1] - heights2[1]]
]

#### AUCs

In [ ]:
fig12, ax12 = plt.subplots(figsize=(7,7))

ax12.bar(tests1, heights1, yerr=CI951, capsize=5)
ax12.set_xlabel(None)
ax12.set_ylabel('Performance (AUROC)')
ax12.set_title("XGBoost trained on Hospital A (2013)")
ax12.grid(linestyle=':', axis='y')
ax12.set_ylim(0.5, 1.0)
fig12.tight_layout()
# plt.savefig(figure_path / 'SIfig2_new.png', dpi=1000)
plt.show()

In [ ]:
fig13, ax13 = plt.subplots(figsize=(7,7))

ax13.bar(tests2, heights2, yerr=CI952, capsize=5)
ax13.set_xlabel(None)
ax13.set_ylabel('Performance (AUROC)')
ax13.set_title("XGBoost trained on Hospital A (2016)")
ax13.grid(linestyle=':', axis='y')
ax13.set_ylim(0.5, 1.0)
fig13.tight_layout()
# plt.savefig(figure_path / 'SIfig2_train2_XGBoost_auc.png', dpi=1000)
plt.show()

In [ ]:
fig14, ax14 = plt.subplots(1, 2, figsize=plots.stdfigsize(nx=2, ny=1))

ax14[0].bar(tests1, heights1, yerr=CI951, capsize=5)
ax14[0].set_xlabel(None)
ax14[0].set_ylabel('Performance (AUROC)')
ax14[0].set_title("XGBoost trained on Hospital A (2013)")
ax14[0].grid(linestyle=':', axis='y')
ax14[0].set_ylim(0.5, 1.0)
ax14[0].text(-0.2, 1.05, "a", transform=ax14[0].transAxes,
               fontsize=30, fontweight='bold', va='top')

ax14[1].bar(tests2, heights2, yerr=CI952, capsize=5)
ax14[1].set_xlabel(None)
ax14[1].set_title("XGBoost trained on Hospital A (2016)")
ax14[1].grid(linestyle=':', axis='y')
ax14[1].set_ylim(0.5, 1.0)
ax14[1].text(-0.2, 1.05, "b", transform=ax14[1].transAxes,
               fontsize=30, fontweight='bold', va='top')

fig14.tight_layout()
# plt.savefig(figure_path / 'SIfig2.png', dpi=1000)
# plt.savefig(figure_path / 'SIfig2.pdf', dpi=1000)
plt.show()

#### Calibration plots

In [ ]:
fig15, ax15 = plt.subplots(2, 2, figsize=(15,15))

# Hospital A (2013) on Hospital A (2016)
mean_fop_1_2 = np.mean(fops_1_2, axis=0)
fop_1_2_CI95 = [np.percentile(fops_1_2, 2.5, axis=0),
                np.percentile(fops_1_2, 97.5, axis=0)]

ax15[0,0].plot(mean_mpv, mean_fop_1_2, marker="o")
ax15[0,0].fill_between(
    mean_mpv, fop_1_2_CI95[0], fop_1_2_CI95[1], alpha=0.2)
ax15[0,0].plot([0, 1], [0, 1], linestyle="--", color="k", alpha=0.8)

ax15[0,0].tick_params(axis='x', labelsize=18)
ax15[0,0].tick_params(axis='y', labelsize=18)
ax15[0,0].set_ylabel("True Positive Rate", size=18)
ax15[0,0].set_title("Train 1, Test 2", size=16)
ax15[0,0].set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax15[0,0].grid(linestyle=':')


# Hospital A (2013) on Hospital B (2017-18)
mean_fop_1_3 = np.mean(fops_1_3, axis=0)
fop_1_3_CI95 = [np.percentile(fops_1_3, 2.5, axis=0),
                np.percentile(fops_1_3, 97.5, axis=0)]

ax15[0,1].plot(mean_mpv, mean_fop_1_3, marker="o")
ax15[0,1].fill_between(
    mean_mpv, fop_1_3_CI95[0], fop_1_3_CI95[1], alpha=0.2)
ax15[0,1].plot([0, 1], [0, 1], linestyle="--", color="k", alpha=0.8)

ax15[0,1].tick_params(axis='x', labelsize=18)
ax15[0,1].tick_params(axis='y', labelsize=18)
ax15[0,1].set_title("Train 1, Test 3", size=16)
ax15[0,1].set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax15[0,1].grid(linestyle=':')


# Hospital A (2016) on Hospital A (2013)
mean_fop_2_1 = np.mean(fops_2_1, axis=0)
fop_2_1_CI95 = [np.percentile(fops_2_1, 2.5, axis=0),
                np.percentile(fops_2_1, 97.5, axis=0)]

ax15[1,0].plot(mean_mpv, mean_fop_2_1, marker="o")
ax15[1,0].fill_between(
    mean_mpv, fop_2_1_CI95[0], fop_2_1_CI95[1], alpha=0.2)
ax15[1,0].plot([0, 1], [0, 1], linestyle="--", color="k", alpha=0.8)

ax15[1,0].tick_params(axis='x', labelsize=18)
ax15[1,0].tick_params(axis='y', labelsize=18)
ax15[1,0].set_xlabel("Mean estimated probability", size=18)
ax15[1,0].set_ylabel("Fraction of positive label", size=18)
ax15[1,0].set_title("Train 2, Test 1", size=16)
ax15[1,0].set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax15[1,0].grid(linestyle=':')


# Hospital A (2016) on Hospital B (2017-18)
mean_fop_2_3 = np.mean(fops_2_3, axis=0)
fop_2_3_CI95 = [
    np.percentile(fops_2_3, 2.5, axis=0),
    np.percentile(fops_2_3, 97.5, axis=0)
    ]

ax15[1,1].plot(mean_mpv, mean_fop_2_3, marker="o")
ax15[1,1].fill_between(
    mean_mpv, fop_2_3_CI95[0], fop_2_3_CI95[1], alpha=0.2)
ax15[1,1].plot([0, 1], [0, 1], linestyle="--", color="k", alpha=0.8)

ax15[1,1].tick_params(axis='x', labelsize=18)
ax15[1,1].tick_params(axis='y', labelsize=18)
ax15[1,1].set_xlabel("Mean estimated probability", size=18)
ax15[1,1].set_title("Train 2, Test 3", size=16)
ax15[1,1].set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax15[1,1].grid(linestyle=':')

# plt.savefig("Calibration_generalization.pdf")
plt.show()

## Testing on MIMIC III

In [ ]:
with open(basedir / "Development_notebooks" /  "hyperparameters" / "bilateral_infiltrates_model_hyperparams.json", "r") as bilat:
    cxr_model_hyperparams = json.load(bilat)

cxr_model_hyperparams['base_score'] = float(cxr_model_hyperparams['base_score'])    
cxr_model_hyperparams['n_estimators'] = int(cxr_model_hyperparams['n_estimators'])
cxr_model_hyperparams['max_depth'] = int(cxr_model_hyperparams['max_depth'])
cxr_model_hyperparams['learning_rate'] = float(cxr_model_hyperparams['learning_rate'])
cxr_model_hyperparams['gamma'] = float(cxr_model_hyperparams['gamma'])
cxr_model_hyperparams['min_child_weight'] = float(cxr_model_hyperparams['min_child_weight'])
cxr_model_hyperparams['max_delta_step'] = float(cxr_model_hyperparams['max_delta_step'])
cxr_model_hyperparams['subsample'] = float(cxr_model_hyperparams['subsample'])

In [ ]:
X5 = training_data[cols[which][0]].to_numpy()
Y5 = training_data[cols[which][1]].to_numpy()

In [ ]:
# Vectorize
vect = CountVectorizer(
    tokenizer=tokenizer_better,
    ngram_range=(1, 2),
    max_features=200
    )

vect.fit(X5)

bilat_infilt_vect = vect.transform(X5).toarray()

bilateral_infiltrates_model = XGBClassifier(
    **cxr_model_hyperparams,
    tree_method='hist',
    random_state=0
    )

bilateral_infiltrates_model.fit(bilat_infilt_vect, Y5)

# # Write vect and bilateral_infiltrates_model to disk
# import pickle

# with open("src/bilateral_infiltrates_model.pickle", "wb") as bilat:
#     pickle.dump(bilateral_infiltrates_model, bilat)
    
# with open("src/bilateral_infiltrates_model_vectorizer.pkl", "wb") as bilat:
#     pickle.dump(vect, bilat)

In [ ]:
tprs_mimic = []
aucs_mimic = []
fops_mimic = []
preds_mimic = []

for i in tqdm(range(n_boot)):
    # This line resamples the data, WITH replacement
    boot_mimic3 = mimic_iii.sample(
        n=len(mimic_iii),
        replace=True,
        axis=0
        )
    
    encounters = boot_mimic3['encounter_id'].unique()
    
    cv = KFold()
    output1, output2, output3, output4, output5 = Parallel(n_jobs=5)(delayed(custom_cv_not_train)(
        bilateral_infiltrates_model,
        boot_mimic3,
        cols2,
        which2,
        encounters,
        vect,
        test_index,
        mean_fpr,
        mean_mpv
        ) for test_index, _ in cv.split(encounters))
    
    aucs_mimic.append(np.mean([output1[0], output2[0], output3[0], output4[0], output5[0]]))
    preds_mimic.extend(output1[1])
    preds_mimic.extend(output2[1])
    preds_mimic.extend(output3[1])
    preds_mimic.extend(output4[1])
    preds_mimic.extend(output5[1])
    tprs_mimic.append(np.mean([output1[2], output2[2], output3[2], output4[2], output5[2]], axis=0))
    fops_mimic.append(np.mean([output1[3], output2[3], output3[3], output4[3], output5[3]], axis=0))

#### ROC curve - test bilateral infiltrates model on MIMIC III

In [ ]:
mean_tpr_mimic = np.mean(tprs_mimic, axis=0)
mean_tpr_mimic[-1] = 1.0
tpr_mimic_CI95 = [
    np.percentile(tprs_mimic, 2.5, axis=0),
    np.percentile(tprs_mimic, 97.5, axis=0)
    ]

mean_auc_mimic = np.mean(aucs_mimic)
auc_mimic_CI95 = [
    np.percentile(aucs_mimic, 2.5),
    np.percentile(aucs_mimic, 97.5)
    ]

In [ ]:
fig16, ax16 = plt.subplots(figsize=(7,7))

ax16.plot(
    mean_fpr,
    mean_tpr_mimic,
    label=f"Mean AUROC (95%CI): {mean_auc_mimic:.2f}\n({auc_mimic_CI95[0]:.2f}-{auc_mimic_CI95[1]:.2f})"
    )
ax16.fill_between(
    mean_fpr,
    tpr_mimic_CI95[0],
    tpr_mimic_CI95[1],
    alpha=0.2
    )

ax16.plot([0, 1], [0, 1], linestyle="--", color="k", alpha=0.8)

ax16.set_ylabel("True Positive Rate")
ax16.set_xlabel("False Positive Rate")
ax16.set_title("Bilateral infiltrates model on MIMIC (2001-12)")
ax16.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax16.grid(linestyle=':')
ax16.legend(loc='best', frameon=False)

fig16.tight_layout()
plt.show()

#### Calibration curve

In [ ]:
mean_fop_mimic = np.mean(fops_mimic, axis=0)
fop_mimic_CI95 = [
    np.percentile(fops_mimic, 2.5, axis=0),
    np.percentile(fops_mimic, 97.5, axis=0)
    ]

In [ ]:
fig17, ax17 = plt.subplots(figsize=(7,7))

ax17.plot(mean_mpv, mean_fop_mimic, marker="o")
ax17.fill_between(
    mean_mpv, fop_mimic_CI95[0], fop_mimic_CI95[1], alpha=0.2)
ax17.plot([0, 1], [0, 1], linestyle="--", color="k", alpha=0.8)

ax17.tick_params(axis='x', labelsize=18)
ax17.tick_params(axis='y', labelsize=18)
ax17.set_xlabel("Mean predicted probability", size=18)
ax17.set_ylabel("Fraction of positive labels", size=18)
ax17.set_title("Bilateral infiltrates model on MIMIC III", size=16)
ax17.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax17.grid(linestyle=':')

plt.show()

#### SHAP values

In [ ]:
vect_mimic = vect.transform(X4).toarray()

In [ ]:
explainer = shap.explainers.Tree(
    bilateral_infiltrates_model,
    vect_mimic,
    feature_perturbation='interventional',
    feature_names=vect.get_feature_names_out(),
    model_output='probability'
    )

shap_values = explainer(vect_mimic)

In [ ]:
shap.summary_plot(shap_values, max_display=15, show=False)

axis1 = plt.gca()
axis1.set_title("Bilateral Infiltrates Model on MIMIC (2001-12)", fontsize=20)
axis1.set_xlabel(None)
axis1.tick_params(axis='both', which='both', labelsize=18)
axis1.set_xlim(-0.5, 0.7)

cb1 = plt.gcf().get_axes()[1]
cb1.set_yticklabels(['0', str(vect_mimic.max())])
cb1.set_ylabel("Word count in report", size=20)
cb1.tick_params(labelsize=20)

plt.tight_layout()
# plt.savefig(figure_path / "SIfig1.png", dpi=1000)

In [ ]:
shap.waterfall_plot(shap_values[25], max_display=15, show=False)

axis5 = plt.gca()
axis5.set_title("SHAP values for sample 25", fontsize=20)

for eje6 in plt.gcf().get_axes():
    eje6.tick_params(axis='both', which='both', labelsize=14)

plt.tight_layout()
plt.show()
plt.close('all')

In [ ]:
mimic_iii.loc[25]

#### Interrater agreement

In [ ]:
fig18, ax18 = plt.subplots(2,3, figsize=plots.stdfigsize(nx=3, ny=2))

# Label frequency Hospital B (2017-18)
ax18[0,0].hist(segmented_cohort3_original.score_final)
ax18[0,0].set_ylabel("Counts")
ax18[0,0].set_title("Hospital B (2017-18) label frequency")
ax18[0,0].grid(linestyle=':')

# Label frequency Hospital A (2013)
ax18[0,1].hist(segmented.cxr_score)
ax18[0,1].set_title("Hospital A (2013) label frequency")
ax18[0,1].grid(linestyle=':')

# Hospital A (2013) on Hospital B (2017-18)
ax18[0,2].hist(preds_1_3, bins=10)
ax18[0,2].set_title("Train Hospital A (2013), Test Hospital B (2017-18) XGBoost probabilites")
ax18[0,2].grid(linestyle=':')


# Label frequency Hospital B (2017-18)
ax18[1,0].hist(segmented_cohort3_original.score_final)
ax18[1,0].set_ylabel("Counts")
ax18[1,0].set_title("Hospital B (2017-18) label frequency")
ax18[1,0].set_xlabel("Labels")
ax18[1,0].grid(linestyle=':')

# Label frequency Hospital A (2016)
ax18[1,1].hist(segmented_cohort2.score)
ax18[1,1].set_title("Hospital A (2016) label frequency")
ax18[1,1].set_xlabel("Labels")
ax18[1,1].grid(linestyle=':')

# Hospital A (2016) on Hospital B (2017-18)
ax18[1,2].hist(preds_2_3, bins=10)
ax18[1,2].set_xlabel("Estimated probabilities")
ax18[1,2].set_title("Train Hospital A (2016), Test Hospital B (2017-18) XGBoost probabilites")
ax18[1,2].grid(linestyle=':')

fig18.tight_layout()
plt.show()

Looks like an easy explanation for XGBoost doing better in predicting Hospital B (2017-18) when trained on Hospital A (2016) is simply the greater similarity in class/label distribution

In [ ]:
l1, l2 = 0.1, 0.9

In [ ]:
# Predictions
mimic_iii['cxr_score_probability'] = bilateral_infiltrates_model.predict_proba(vect_mimic)[:, 1]

Fraction disagree

In [ ]:
# Subsetting/bucketing table by XGBoost-outputted probability ranges
# GOTTA SPECIFY WHICH COLUMNS TO DO THE DROPNA FOR
agg_disagree = []
r = mimic_iii['cxr_score_probability'] < l1
s = mimic_iii['cxr_score_probability'] >= l1
t = mimic_iii['cxr_score_probability'] < l2
u = mimic_iii['cxr_score_probability'] >= l2

for z in tqdm(range(n_boot)):
    lows1 = mimic_iii.loc[r].sample(n=r.sum(), replace=True, axis=0)
    intermediates1 = mimic_iii.loc[s&t].sample(n=(s&t).sum(), replace=True, axis=0)
    highs1 = mimic_iii.loc[u].sample(n=u.sum(), replace=True, axis=0)
    boot_mimic_iii = pd.concat([lows1, intermediates1, highs1], ignore_index=True)
    
    a5 = boot_mimic_iii['cxr_score_probability'] < l1
    temp = boot_mimic_iii[a5].dropna(subset=['eryn_bl_infiltrates_(1=yes)', 'curt_bl_infiltrates_(1=yes)', 'seg_cxr_text'])
    disagreements = 0
    for idx, row in temp.iterrows():
        disagreements += int(row["eryn_bl_infiltrates_(1=yes)"] != row["curt_bl_infiltrates_(1=yes)"])
    
    agg_disagree.append({'model_confidence': 'High\nconfidence\nNo', 'disagreement': disagreements/len(temp), 'model': "Bilateral infiltrates model"})


    a6 = boot_mimic_iii['cxr_score_probability'] >= l1
    a7 = boot_mimic_iii['cxr_score_probability'] < l2
    temp = boot_mimic_iii[a6&a7].dropna(subset=['eryn_bl_infiltrates_(1=yes)', 'curt_bl_infiltrates_(1=yes)', 'seg_cxr_text'])
    disagreements = 0
    for idx, row in temp.iterrows():
        disagreements += int(row["eryn_bl_infiltrates_(1=yes)"] != row["curt_bl_infiltrates_(1=yes)"])

    agg_disagree.append({'model_confidence': "Low confidence", 'disagreement': disagreements/len(temp), 'model': "Bilateral infiltrates model"})


    a8 = boot_mimic_iii['cxr_score_probability'] >= l2
    temp = boot_mimic_iii[a8].dropna(subset=['eryn_bl_infiltrates_(1=yes)', 'curt_bl_infiltrates_(1=yes)', 'seg_cxr_text'])
    disagreements = 0
    for idx, row in temp.iterrows():
        disagreements += int(row["eryn_bl_infiltrates_(1=yes)"] != row["curt_bl_infiltrates_(1=yes)"])

    agg_disagree.append({'model_confidence': "High\nconfidence\nYes", 'disagreement': disagreements/len(temp), 'model': "Bilateral infiltrates model"})

In [ ]:
disagreement_df = pd.DataFrame(agg_disagree)

In [ ]:
disagreement_df.groupby('model_confidence')['disagreement'].mean()

In [ ]:
fig19, ax19 = plt.subplots(figsize=plots.stdfigsize())

sns.barplot(
    x='model_confidence',
    y='disagreement',
    data=disagreement_df,
    capsize=.2,
    hue='model_confidence',
    ax=ax19
    )

ax19.annotate(f"n={r.sum()}", (-0.20, 0.06))
ax19.annotate(f"n={(s&t).sum()}", (0.80, 0.1))
ax19.annotate(f"n={u.sum()}", (1.80, 0.17))

ax19.set_ylabel("Mean fraction disagreed")
ax19.grid(linestyle=':', axis='y')
ax19.set_xlabel("")
ax19.set_ylim(0, 0.4)
# ax22.legend()

fig19.tight_layout()
# plt.savefig(figure_path / 'fig3_disagreement.png', dpi=1000)
# plt.savefig(figure_path / 'fig3_disagreement.pdf', dpi=1000)
plt.show()

Confusion matrix

In [ ]:
# mimic_iii['predicted'] = mimic_iii['cxr_score_probability'] >= threshold
# If threshold == 0.5, the line above and the line below do the exact same thing
mimic_iii['predicted'] = bilateral_infiltrates_model.predict(vect_mimic)

cf = confusion_matrix(Y4, mimic_iii['predicted'])

strings = np.asarray([['True negatives\n', 'False positives\n'],
                      ['False negatives\n', 'True positives\n']])

labels = (np.asarray(["{0} {1:.0f}".format(string, value)
                      for string, value in zip(strings.flatten(),
                                               cf.flatten())])
         ).reshape(2, 2)

fig20, ax20 = plt.subplots(figsize=plots.stdfigsize())
sns.heatmap(cf, fmt='', annot=labels, cmap='Blues', cbar=False, ax=ax20)
ax20.set_ylabel("Ground truth")
ax20.set_xlabel("Bilateral Infiltrates Model adjudicated")
ax20.tick_params(axis='both', bottom=False, left=False,
                labelbottom=False, labelleft=False)

fig20.tight_layout()
# plt.savefig(figure_path / 'fig3_cf.png')
plt.show()

In [ ]:
fig21, ax21 = plt.subplots(2, 2, figsize=plots.stdfigsize(nx=2, ny=2))

# ROC curve
ax21[0,0].plot(
    mean_fpr,
    mean_tpr_mimic,
    label=f"Mean AUROC: {mean_auc_mimic:.2f}"
    )
ax21[0,0].fill_between(
    mean_fpr,
    tpr_mimic_CI95[0],
    tpr_mimic_CI95[1],
    alpha=0.2
    )
ax21[0,0].plot([0, 1], [0, 1], linestyle="--", color="k", alpha=0.8, label="No-skill line")

# Calibration curve
ax21[0,1].plot(mean_mpv, mean_fop_mimic, marker="o")
ax21[0,1].fill_between(
    mean_mpv, fop_mimic_CI95[0], fop_mimic_CI95[1], alpha=0.2)
ax21[0,1].plot([0, 1], [0, 1], linestyle="--", color="k", alpha=0.8, label="Perfectly calibrated")

# Confusion matrix
sns.heatmap(cf, fmt='', annot=labels, cmap='Blues', cbar=False, ax=ax21[1,0])

# Interrater disagreement
sns.barplot(x='model_confidence', y='disagreement', hue='model_confidence',
            data=disagreement_df, capsize=.2, ax=ax21[1,1])

# Plot properties
ax21[0,0].set_ylabel("True Positive Rate")
ax21[0,0].set_xlabel("False Positive Rate")
ax21[0,0].set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax21[0,0].grid(linestyle=':')
ax21[0,0].legend(loc='best', frameon=False)
ax21[0,0].text(-0.2, 1.05, "a", transform=ax21[0,0].transAxes,
               fontsize=30, fontweight='bold', va='top')

ax21[0,1].tick_params(axis='x', labelsize=18)
ax21[0,1].tick_params(axis='y', labelsize=18)
ax21[0,1].set_xlabel("Mean predicted probability")
ax21[0,1].set_ylabel("Fraction of positive labels")
ax21[0,1].set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])
ax21[0,1].grid(linestyle=':')
ax21[0,1].text(-0.2, 1.05, "b", transform=ax21[0,1].transAxes,
               fontsize=30, fontweight='bold', va='top')

ax21[1,0].set_ylabel("Ground truth")
ax21[1,0].set_xlabel("Bilateral Infiltrates Model adjudicated")
ax21[1,0].tick_params(axis='both', bottom=False, left=False,
                labelbottom=False, labelleft=False)
ax21[1,0].text(-0.2, 1.05, "c", transform=ax21[1,0].transAxes,
               fontsize=30, fontweight='bold', va='top')

ax21[1,1].annotate(f"n={r.sum()}", (-0.20, 0.06))
ax21[1,1].annotate(f"n={(s&t).sum()}", (0.8, 0.10))
ax21[1,1].annotate(f"n={u.sum()}", (1.8, 0.17))
ax21[1,1].set_ylabel("Mean fraction disagreed")
ax21[1,1].grid(linestyle=':', axis='y')
ax21[1,1].set_xlabel("")
ax21[1,1].set_ylim(0.0, 0.4)
ax21[1,1].text(-0.2, 1.05, "d", transform=ax21[1,1].transAxes,
               fontsize=30, fontweight='bold', va='top')


fig21.tight_layout()
# plt.savefig(figure_path / "fig3.png", dpi=1000)
# plt.savefig(figure_path / "fig3.pdf", dpi=1000)
plt.show()

#### SI Figure on different choices for cutoff

In [ ]:
# try different thresholds for the probability and see how the F1 score changes
f1_scores = []
f1_thresholds = np.linspace(0, 1, 1000)

for threshold in f1_thresholds:
    predictions = mimic_iii['cxr_score_probability'] >= threshold
    f1_scores.append(f1_score(mimic_iii['curt_bl_infiltrates_(1=yes)'], predictions))

In [ ]:
# Get index of maximum value of scores
max_f1_score = max(f1_scores)
max_index = f1_scores.index(max_f1_score)
max_f1_threshold = f1_thresholds[max_index]
print(f"Maximum F1 Score: {max_f1_score} at threshold: {max_f1_threshold}")

In [ ]:
mimic_iii['predicted_f1'] = mimic_iii['cxr_score_probability'] >= max_f1_threshold

cf_f1 = confusion_matrix(mimic_iii['curt_bl_infiltrates_(1=yes)'], mimic_iii['predicted_f1'])

strings = np.asarray([['True negatives\n', 'False positives\n'],
                      ['False negatives\n', 'True positives\n']])

labels_f1 = (np.asarray(["{0} {1:.0f}".format(string, value)
                      for string, value in zip(strings.flatten(),
                                               cf_f1.flatten())])
             ).reshape(2, 2)

In [ ]:
# try different thresholds for the probability and see how the accuracy changes
accuracy_scores = []
accuracy_thresholds = np.linspace(0, 1, 1000)

for threshold in accuracy_thresholds:
    predictions = mimic_iii['cxr_score_probability'] >= threshold
    accuracy_scores.append(accuracy_score(mimic_iii['curt_bl_infiltrates_(1=yes)'], predictions))

In [ ]:
# Get index of maximum value of scores
max_accuracy_score = max(accuracy_scores)
max_index = accuracy_scores.index(max_accuracy_score)
max_accuracy_threshold = accuracy_thresholds[max_index]
print(f"Maximum Accuracy: {max_accuracy_score} at threshold: {max_accuracy_threshold}")

In [ ]:
mimic_iii['predicted_accuracy'] = mimic_iii['cxr_score_probability'] >= max_accuracy_threshold

cf_accuracy = confusion_matrix(mimic_iii['curt_bl_infiltrates_(1=yes)'], mimic_iii['predicted_accuracy'])

strings = np.asarray([['True negatives\n', 'False positives\n'],
                      ['False negatives\n', 'True positives\n']])

labels_accuracy = (np.asarray(["{0} {1:.0f}".format(string, value)
                      for string, value in zip(strings.flatten(),
                                               cf_accuracy.flatten())])
                   ).reshape(2, 2)

In [ ]:
fig22, ax22 = plt.subplots(1, 2, figsize=plots.stdfigsize(nx=2, ny=1))

sns.heatmap(cf_f1, fmt='', annot=labels_f1, cmap='Blues', cbar=False, ax=ax22[0])
ax22[0].set_ylabel("Ground truth")
ax22[0].set_xlabel("Bilateral Infiltrates Model adjudicated")
ax22[0].set_title("Optimal F1 score")
ax22[0].tick_params(axis='both', bottom=False, left=False,
                labelbottom=False, labelleft=False)
ax22[0].text(-0.2, 1.05, "a", transform=ax22[0].transAxes,
               fontsize=30, fontweight='bold', va='top')

sns.heatmap(cf_accuracy, fmt='', annot=labels_accuracy, cmap='Blues', cbar=False, ax=ax22[1])
ax22[1].set_ylabel("Ground truth")
ax22[1].set_xlabel("Bilateral Infiltrates Model adjudicated")
ax22[1].set_title("Optimal accuracy")
ax22[1].tick_params(axis='both', bottom=False, left=False,
                labelbottom=False, labelleft=False)
ax22[1].text(-0.2, 1.05, "b", transform=ax22[1].transAxes,
               fontsize=30, fontweight='bold', va='top')

fig22.tight_layout()
# plt.savefig(figure_path / 'SIfig3.png', dpi=1000)
# plt.savefig(figure_path / 'SIfig3.pdf', dpi=1000)
plt.show()

#### How would all of this look when raters agree vs. when they disagree?

In [ ]:
mimic_iii_agree = mimic_iii.loc[(mimic_iii['curt_bl_infiltrates_(1=yes)'] == mimic_iii['eryn_bl_infiltrates_(1=yes)'])]
mimic_iii_disagree = mimic_iii.loc[(mimic_iii['curt_bl_infiltrates_(1=yes)'] != mimic_iii['eryn_bl_infiltrates_(1=yes)'])]

In [ ]:
len(mimic_iii_agree), len(mimic_iii_disagree)

In [ ]:
print(f"AUROC agree: {roc_auc_score(mimic_iii_agree['curt_bl_infiltrates_(1=yes)'], mimic_iii_agree['cxr_score_probability'])}")
print(f"AUROC disagree: {roc_auc_score(mimic_iii_disagree['curt_bl_infiltrates_(1=yes)'], mimic_iii_disagree['cxr_score_probability'])}")

print(f"Accuracy agree: {accuracy_score(mimic_iii_agree['curt_bl_infiltrates_(1=yes)'], mimic_iii_agree['predicted'])}")
print(f"Accuracy disagree: {accuracy_score(mimic_iii_disagree['curt_bl_infiltrates_(1=yes)'], mimic_iii_disagree['predicted'])}")

In [ ]:
from sklearn.calibration import CalibrationDisplay

prob_true, prob_pred = calibration_curve(mimic_iii_agree['curt_bl_infiltrates_(1=yes)'], mimic_iii_agree['cxr_score_probability'], n_bins=10)
CalibrationDisplay(prob_true, prob_pred, mimic_iii_agree['cxr_score_probability']).plot()
plt.show()

In [ ]:
prob_true, prob_pred = calibration_curve(mimic_iii_disagree['curt_bl_infiltrates_(1=yes)'], mimic_iii_disagree['cxr_score_probability'], n_bins=10)
CalibrationDisplay(prob_true, prob_pred, mimic_iii_disagree['cxr_score_probability']).plot()
plt.show()

The real deal: the interrater disagreement graph

In [ ]:
low_conf = mimic_iii.loc[(mimic_iii['cxr_score_probability'] >= l1) & (mimic_iii['cxr_score_probability'] < l2)]
high_conf_no = mimic_iii.loc[mimic_iii['cxr_score_probability'] < l1]
high_conf_yes = mimic_iii.loc[mimic_iii['cxr_score_probability'] >= l2]

In [ ]:
len(low_conf), len(high_conf_no), len(high_conf_yes)

In [ ]:
print(f"Accuracy high confidence no: {accuracy_score(high_conf_no['curt_bl_infiltrates_(1=yes)'], high_conf_no['predicted'])}")
print(f"Accuracy low confidence: {accuracy_score(low_conf['curt_bl_infiltrates_(1=yes)'], low_conf['predicted'])}")
print(f"Accuracy high confidence yes: {accuracy_score(high_conf_yes['curt_bl_infiltrates_(1=yes)'], high_conf_yes['predicted'])}")

I remember looking at the reports, and wondering about how frequently would the model give a low probability to a report that was rated "yes" by both raters. Or a high probability to a report that was rated "no" by both raters.

In [ ]:
# I remember looking at the reports, and wondering about how frequently would the model give a low probability to a report that was rated "yes" by both raters. Or a high probability to a report that was rated "no" by both raters. This is the graph that answers that question.
low_conf_agree = mimic_iii_agree.loc[(mimic_iii_agree['cxr_score_probability'] >= l1) & (mimic_iii_agree['cxr_score_probability'] < l2)]
high_conf_no_agree = mimic_iii_agree.loc[mimic_iii_agree['cxr_score_probability'] < l1]
high_conf_yes_agree = mimic_iii_agree.loc[mimic_iii_agree['cxr_score_probability'] >= l2]

low_conf_disagree = mimic_iii_disagree.loc[(mimic_iii_disagree['cxr_score_probability'] >= l1) & (mimic_iii_disagree['cxr_score_probability'] < l2)]
high_conf_no_disagree = mimic_iii_disagree.loc[mimic_iii_disagree['cxr_score_probability'] < l1]
high_conf_yes_disagree = mimic_iii_disagree.loc[mimic_iii_disagree['cxr_score_probability'] >= l2]

In [ ]:
print(f"Accuracy high confidence no: {accuracy_score(high_conf_no_agree['curt_bl_infiltrates_(1=yes)'], high_conf_no_agree['predicted'])}")
print(f"Accuracy low confidence: {accuracy_score(low_conf_agree['curt_bl_infiltrates_(1=yes)'], low_conf_agree['predicted'])}")
print(f"Accuracy high confidence yes: {accuracy_score(high_conf_yes_agree['curt_bl_infiltrates_(1=yes)'], high_conf_yes_agree['predicted'])}")
print("\n")
print(f"Accuracy high confidence no: {accuracy_score(high_conf_no_disagree['curt_bl_infiltrates_(1=yes)'], high_conf_no_disagree['predicted'])}")
print(f"Accuracy low confidence: {accuracy_score(low_conf_disagree['curt_bl_infiltrates_(1=yes)'], low_conf_disagree['predicted'])}")
print(f"Accuracy high confidence yes: {accuracy_score(high_conf_yes_disagree['curt_bl_infiltrates_(1=yes)'], high_conf_yes_disagree['predicted'])}")